In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
import pandas as pd
import random
import scipy.signal as ss
import sys
sys.path.append(module_path)

import tools.data_reader_apd as dr_a
import tools.data_reader_ascertain as dr_asc
import tools.data_reader_wesad as dr_w
import tools.data_reader_popane as dr_p
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize
from xgboost import XGBClassifier

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)

c:\Users\zhoux\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from itertools import combinations

temp_a, _ = train.Train_APD.get_apd_data_ranking([train.Metrics.BPM], phases=dr_a.Phases.PHASES_LIST, normalize=False)
idx = temp_a[temp_a["bpm"] > 200].index 
invalid_apd_subjects = set(temp_a["subject"].iloc[idx].tolist())
idx = temp_a[temp_a["bpm"] < 35].index 
invalid_apd_subjects.update(set(temp_a["subject"].iloc[idx].tolist()))

metrics_list = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    # train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR,
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE, 
]

metrics_combinations = {
    1: None,
    2: None,
    3: None,
    4: None,
    5: None,
    6: None,
    7: None,
    8: None,
}

for i in list(metrics_combinations.keys()):
    metrics_combinations[i] = [list(metrics) for metrics in combinations(metrics_list, i)]

threshold = "fixed"

In [13]:
# LOAD TRAIN AND TEST DATA
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dt)


model_phases = [
    [
        "Baseline_Rest", 
        "BugBox_Relax", "BugBox_Anticipate", "BugBox_Exposure", "BugBox_Break",
        "Speech_Relax", "Speech_Anticipate", "Speech_Exposure", "Speech_Break"
    ],
    ["Baseline_Rest"], ["BugBox_Relax"], ["BugBox_Anticipate"], ["BugBox_Exposure"], ["BugBox_Break"], 
    ["Speech_Relax"], ["Speech_Anticipate"], ["Speech_Exposure"], ["Speech_Break"]
]

# anxiety_label_type = "Anxiety"
anxiety_label_type = None



for phases in model_phases:
    print(f"PHASES: {phases}")
    results_apd = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}
    for i in list(metrics_combinations.keys()):
        for metrics in metrics_combinations[i]:
            # print(f"METRICS: {metrics}")
            x, y = train.Train_APD.get_apd_data_ranking(metrics, phases, verbose=False, anxiety_label_type=anxiety_label_type, threshold=threshold, normalize=True)
            x = x.drop(["phaseId"], axis=1)
            # drop subjects with noisy data
            x = x[~x["subject"].isin(invalid_apd_subjects)]
            y = y[~y["subject"].isin(invalid_apd_subjects)]

            if anxiety_label_type is not None:
                x.drop(labels=["anxietyGroup"], axis=1)

            model = LinearRegression()
            model.fit(x, y)
            # print(model.intercept_) 
            # print(model.coef_)
            # print(f"R2: {model.score(x, y)}\n")
            results_apd[i].append([model.score(x, y), metrics])
    
    results_apd_sorted = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}
    for i in list(results_apd.keys()):
        results_apd_sorted[i] = sorted(results_apd[i], key=lambda x: x[0], reverse=True)

    for i in list(results_apd_sorted.keys()):
        for j in range(5):
            try:   
                print(results_apd_sorted[i][j])
            except Exception as e:
                continue
    print("")


PHASES: ['Baseline_Rest', 'BugBox_Relax', 'BugBox_Anticipate', 'BugBox_Exposure', 'BugBox_Break', 'Speech_Relax', 'Speech_Anticipate', 'Speech_Exposure', 'Speech_Break']


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\zhoux\\Desktop\\Projects\\anxiety\\data\\metrics\\APD\\lf_hf_ratio_Baseline_Rest_ha.csv'

In [4]:
# LOAD TRAIN AND TEST DATA
importlib.reload(train)
importlib.reload(dr_w)
importlib.reload(dt)

model_phases = [
    [
        dr_w.Phases.BASE,
        dr_w.Phases.FUN,
        dr_w.Phases.TSST,
        dr_w.Phases.MEDI_1,
        dr_w.Phases.MEDI_2
    ],
    [dr_w.Phases.BASE], [dr_w.Phases.FUN], [dr_w.Phases.TSST], [dr_w.Phases.MEDI_1], [dr_w.Phases.MEDI_2]
]

label_type = "stai"
results_wesad = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}

for phases in model_phases:
    print(f"PHASES: {phases}")
    for i in list(metrics_combinations.keys()):
        for metrics in metrics_combinations[i]:
            # print(f"METRICS: {metrics}")
            x, y = train.Train_WESAD.get_wesad_data(metrics, phases, verbose=False, label_type=label_type, threshold=threshold, normalize=True)
            x = x.drop(["phaseId"], axis=1)

            model = LinearRegression()
            model.fit(x, y)
            # print(model.intercept_) 
            # print(model.coef_)
            # print(f"R2: {model.score(x, y)}\n")
            results_wesad[i].append([model.score(x, y), metrics])

    results_wesad_sorted = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}
    for i in list(results_wesad.keys()):
        results_wesad_sorted[i] = sorted(results_wesad[i], key=lambda x: x[0], reverse=True)

    for i in list(results_apd_sorted.keys()):
        print(f"Top R2 values for any combination of {i} metrics:")
        for j in range(5):
            try:   
                print(results_wesad_sorted[i][j])
            except Exception as e:
                continue
    print("")


PHASES: ['Base', 'Fun', 'TSST', 'Medi_1', 'Medi_2']
Top R2 values for any combination of 1 metrics:
[0.5122569306160616, ['hf_rr']]
[0.5100262613944904, ['bpm']]
[0.5022178217658921, ['ibi']]
[0.5008210073575816, ['lf_rr']]
[0.5002398965992731, ['rmssd']]
Top R2 values for any combination of 2 metrics:
[0.5271579743368457, ['hf_rr', 'lf_rr']]
[0.5256980044471371, ['bpm', 'hf_rr']]
[0.517180885644235, ['ibi', 'hf_rr']]
[0.5143341590027202, ['bpm', 'rmssd']]
[0.5137187027971639, ['hf_rr', 'mean_SCL']]
Top R2 values for any combination of 3 metrics:
[0.5373352100016138, ['bpm', 'hf_rr', 'lf_rr']]
[0.5330244284361745, ['ibi', 'hf_rr', 'lf_rr']]
[0.5291362842932722, ['bpm', 'rmssd', 'hf_rr']]
[0.5288446869213936, ['hf_rr', 'lf_rr', 'SCR_rate']]
[0.5284354174871446, ['sdnn', 'hf_rr', 'lf_rr']]
Top R2 values for any combination of 4 metrics:
[0.5408634240670634, ['bpm', 'hf_rr', 'lf_rr', 'SCR_rate']]
[0.5405393406184493, ['bpm', 'rmssd', 'hf_rr', 'lf_rr']]
[0.5383890174313102, ['bpm', 'hf_rr'

In [ ]:
# LOAD TRAIN AND TEST DATA
importlib.reload(train)
importlib.reload(dr_p)
importlib.reload(dt)


popane_phases = {
    "Study1": dr_p.Study1.ALL,
    "Study2": dr_p.Study2.ALL,
    "Study3": dr_p.Study3.ALL,
    # "Study4": dr_p.Study4.ALL,
    "Study5": dr_p.Study5.ALL,
    # "Study6": dr_p.Study6.ALL,
    # "Study7": dr_p.Study7.ALL
}

label_type = "affect"

for study in popane_phases.keys():
    print(f"{study} " + "-"*60)
    results_popane = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}
    phases = popane_phases[study]
    for i in list(metrics_combinations.keys()):
        for metrics in metrics_combinations[i]:
            # print(f"METRICS: {metrics}")
            x, y = train.Train_POPANE.get_popane_data(study, metrics, phases, verbose=False, label_type=label_type, threshold=threshold, normalize=True)
            # x = x.drop(["phaseId"], axis=1)
            nan_idx = x[x.isna().any(axis=1)].index
            x = x.drop(index=nan_idx)
            y = y.drop(index=nan_idx)

            model = LinearRegression()

            model.fit(x, y)
            # print(model.intercept_) 
            # print(model.coef_)
            results_popane[i].append([model.score(x, y), metrics])
    
    results_popane_sorted = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}
    for i in list(results_popane.keys()):
        results_popane_sorted[i] = sorted(results_popane[i], key=lambda x: x[0], reverse=True)

    for i in list(results_popane_sorted.keys()):
        print(f"Top R2 values for any combination of {i} metrics:")
        for j in range(5):
            try:   
                print(results_popane_sorted[i][j])
            except Exception as e:
                continue
    print("")

In [5]:
# LOAD TRAIN AND TEST DATA
importlib.reload(train)
importlib.reload(dr_asc)
importlib.reload(dt)


label_type = dr_asc.SelfReports.AROUSAL
results_asc = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}

for i in list(metrics_combinations.keys()):
    for metrics in metrics_combinations[i]:
        # print(f"METRICS: {metrics}")
        x, y = train.Train_ASCERTAIN.get_ascertain_data(metrics, verbose=False, label_type=label_type, threshold=threshold, normalize=True)
        # x = x.drop(["phaseId"], axis=1)
        nan_idx = x[x.isna().any(axis=1)].index
        x = x.drop(index=nan_idx)
        y = y.drop(index=nan_idx)

        model = LinearRegression()

        model.fit(x, y)
        # print(model.intercept_) 
        # print(model.coef_)
        results_asc[i].append([model.score(x, y), metrics])

results_asc_sorted = {1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: []}
for i in list(results_asc.keys()):
    results_asc_sorted[i] = sorted(results_asc[i], key=lambda x: x[0], reverse=True)

for i in list(results_asc_sorted.keys()):
    print(f"Top R2 values for any combination of {i} metrics:")
    for j in range(5):
        try:   
            print(results_asc_sorted[i][j])
        except Exception as e:
            continue

Top R2 values for any combination of 1 metrics:
[0.5079058785684982, ['sdnn']]
[0.5078910214174017, ['rmssd']]
[0.5035777903434777, ['ibi']]
[0.5035555570297201, ['bpm']]
[0.5035313950363014, ['lf_rr']]
Top R2 values for any combination of 2 metrics:
[0.5103597553272514, ['rmssd', 'mean_SCL']]
[0.5101241030081097, ['sdnn', 'mean_SCL']]
[0.5098285560866915, ['rmssd', 'SCR_rate']]
[0.5096189328861245, ['sdnn', 'SCR_rate']]
[0.509568340857155, ['rmssd', 'ibi']]
Top R2 values for any combination of 3 metrics:
[0.5121564591812823, ['rmssd', 'ibi', 'mean_SCL']]
[0.5119585616969844, ['rmssd', 'ibi', 'SCR_rate']]
[0.5109471050619415, ['ibi', 'sdnn', 'mean_SCL']]
[0.5107620715463146, ['ibi', 'sdnn', 'SCR_rate']]
[0.5104569134781018, ['rmssd', 'mean_SCL', 'SCR_rate']]
Top R2 values for any combination of 4 metrics:
[0.5136995324194444, ['bpm', 'rmssd', 'ibi', 'mean_SCL']]
[0.5133243145268421, ['bpm', 'rmssd', 'ibi', 'SCR_rate']]
[0.5129568611716802, ['rmssd', 'ibi', 'sdnn', 'mean_SCL']]
[0.51275

In [14]:
print("R2 values for APD " + "-"*30)
for i in list(results_apd_sorted.keys()):
    print(f"Top R2 values for any combination of {i} metrics:")
    j = 0
    while j < 5 and j < len(results_apd_sorted[i]):
        print(results_apd_sorted[i][j])
        j += 1
    print("")

print("R2 values for WESAD " + "-"*30)
for i in list(results_wesad_sorted.keys()):
    print(f"Top 5 R2 values for any combination of {i} metrics:")
    j = 0
    while j < 5 and j < len(results_wesad_sorted[i]):
        print(results_wesad_sorted[i][j])
        j += 1
    print("")

print("R2 values for ASCERTAIN " + "-"*30)
for i in list(results_asc.keys()):
    print(f"Top 5 R2 values for any combination of {i} metrics:")
    j = 0
    while j < 5 and j < len(results_apd_sorted[i]):
        print(results_asc_sorted[i][j])
        j += 1
    print("")

R2 values for APD ------------------------------
Top R2 values for any combination of 1 metrics:
[0.5279478594964077, ['lf_rr']]
[0.5049898498857452, ['rmssd']]
[0.5044044613170494, ['mean_SCL']]
[0.5041032747778251, ['hf_rr']]
[0.5027623868881108, ['SCR_rate']]

Top R2 values for any combination of 2 metrics:
[0.5446068624955609, ['hf_rr', 'lf_rr']]
[0.5348282877861277, ['bpm', 'lf_rr']]
[0.5329478007440787, ['rmssd', 'lf_rr']]
[0.5325962470562027, ['ibi', 'lf_rr']]
[0.5317282686017705, ['lf_rr', 'mean_SCL']]

Top R2 values for any combination of 3 metrics:
[0.5636696846832481, ['hf_rr', 'lf_rr', 'mean_SCL']]
[0.552257579183979, ['ibi', 'hf_rr', 'lf_rr']]
[0.5515551271135899, ['bpm', 'hf_rr', 'lf_rr']]
[0.5485335968588465, ['rmssd', 'hf_rr', 'lf_rr']]
[0.5461008981661358, ['sdnn', 'hf_rr', 'lf_rr']]

Top R2 values for any combination of 4 metrics:
[0.5844879127444198, ['bpm', 'hf_rr', 'lf_rr', 'mean_SCL']]
[0.5807142175330415, ['hf_rr', 'lf_rr', 'mean_SCL', 'SCR_rate']]
[0.57699635076